# Punto 3: Optimización 3D de Estructura Cristalina NdFe₁₂ con Ti

## Objetivo

Encontrar la configuración óptima de **8 átomos de Titanio (Ti)** que sustituyen a átomos de Hierro (Fe) en la estructura cristalina de NdFe₁₂, minimizando la energía del sistema mediante el potencial de Morse.

### Sistema 3D:
- **N posiciones candidatas**: Fe o Ti (coordenadas cristalográficas)
- **16 átomos de Nd**: Fijos (tierras raras)
- **8 átomos de Ti**: A optimizar
- **Total**: (N-8) Fe + 8 Ti + 16 Nd átomos

### Espacio de búsqueda:
$$\\binom{N}{8} \\text{ configuraciones posibles}$$

### Método:
- **Simulated Annealing** con enfriamiento logarítmico: $T(t) = \\frac{c}{\\log(t + t_0)}$
- **64 runs independientes** × 1,000,000 iteraciones
- **Paralelización** con joblib para acelerar cálculos
- **Optimización Numba** (@njit) para cálculo incremental de energía (speedup ~25-1000×)

---
## Sección 1: Setup e Imports

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import sys
import time
from pathlib import Path

# Agregar path para imports de módulos
project_root = Path.cwd().parent
src_path = project_root / 'src' / 'punto3'
sys.path.insert(0, str(src_path))

print(f"Proyecto: {project_root}")
print(f"Módulos punto3: {src_path}")
print("✓ Imports completados")

Proyecto: /home/ge-murillo/CadenasMarkovParcial
Módulos punto3: /home/ge-murillo/CadenasMarkovParcial/src/punto3
✓ Imports completados


In [49]:
# Imports de módulos personalizados
from morse.potential_numba_3d import (
    preparar_morse_params_array,
    get_morse_params_by_names
)
from grid.config_utils_3d import (
    crear_configuracion_inicial_3d,
    get_Ti_positions_3d,
    get_Fe_positions_3d,
    get_Nd_positions_3d
)
from grid.energy_numba_3d import (
    compute_total_energy_3d,
    validate_delta_E_3d
)
from optimization.sa_numba_3d import simulated_annealing_logarithmic_3d
from optimization.parallel_runs_3d import (
    ejecutar_multiples_runs_3d,
    get_best_run,
    get_run_statistics,
    filter_top_runs
)
from analysis.spatial_analysis_numba_3d import (
    analizar_patron_espacial_3d,
    interpretar_patron_3d
)
from visualization.plotting_3d import (
    plot_configuration_3d_plotly,
    plot_projections_2d,
    plot_energy_evolution,
    plot_spatial_metrics_3d,
    plot_multiple_runs_comparison_3d
)

print("✓ Módulos personalizados importados correctamente")

✓ Módulos personalizados importados correctamente


### Cargar Coordenadas Cristalinas

Las coordenadas fueron extraídas del notebook `cristal.ipynb` (corregidas a 96 Fe exactos).

In [50]:
# Coordenadas de átomos de Fe (posiciones candidatas para Fe/Ti)
# IMPORTANTE: Actualiza este string con las coordenadas correctas de tu estructura cristalina
Fe_coords_text = """2.502960 8.592000 1.202000
6.089040 0.000000 1.202000
8.592000 2.502960 1.202000
0.000000 6.089040 1.202000
11.094960 4.296000 1.202000
1.793040 12.888000 1.202000
12.888000 11.094960 1.202000
4.296000 1.793040 1.202000
6.089040 8.592000 1.202000
2.502960 0.000000 1.202000
8.592000 6.089040 1.202000
0.000000 2.502960 1.202000
1.793040 4.296000 1.202000
11.094960 12.888000 1.202000
12.888000 1.793040 1.202000
4.296000 11.094960 1.202000
4.296000 0.000000 0.000000
12.888000 8.592000 0.000000
8.592000 4.296000 0.000000
0.000000 12.888000 0.000000
8.592000 12.888000 0.000000
0.000000 4.296000 0.000000
12.888000 0.000000 0.000000
4.296000 8.592000 0.000000
4.296000 8.592000 2.404000
12.888000 0.000000 2.404000
8.592000 12.888000 2.404000
0.000000 4.296000 2.404000
8.592000 4.296000 2.404000
0.000000 12.888000 2.404000
12.888000 8.592000 2.404000
4.296000 0.000000 2.404000
5.484000 5.484000 0.000000
11.700000 11.700000 0.000000
1.206000 11.700000 0.000000
7.398000 1.206000 0.000000
7.398000 1.206000 2.404000
1.206000 11.700000 2.404000
11.700000 11.700000 2.404000
5.484000 5.484000 2.404000
11.700000 5.484000 2.404000
5.484000 11.700000 2.404000
7.398000 7.398000 2.404000
1.206000 1.206000 2.404000
1.206000 1.206000 0.000000
7.398000 7.398000 0.000000
5.484000 11.700000 0.000000
11.700000 5.484000 0.000000
15.390960 6.089040 1.202000
10.385040 1.793040 1.202000
6.807000 11.094960 1.202000
1.793040 6.807000 1.202000
10.385040 11.094960 1.202000
1.793040 15.390960 1.202000
6.807000 1.793040 1.202000
15.390960 6.807000 1.202000
1.793040 10.385040 1.202000
6.807000 6.089040 1.202000
15.390960 1.793040 1.202000
11.094960 6.807000 1.202000
6.089040 1.793040 1.202000
11.094960 10.385040 1.202000
2.502960 6.807000 1.202000
6.807000 2.502960 1.202000
11.094960 15.390960 1.202000
6.089040 6.807000 1.202000
2.502960 10.385040 1.202000
10.385040 2.502960 1.202000
15.390960 11.094960 1.202000
10.385040 6.089040 1.202000
6.807000 15.390960 1.202000
2.502960 1.793040 1.202000
15.390960 2.502960 1.202000
6.089040 10.385040 1.202000
12.888000 8.592000 2.404000
4.296000 0.000000 0.000000
12.888000 0.000000 2.404000
4.296000 8.592000 0.000000
4.296000 8.592000 2.404000
12.888000 0.000000 0.000000
8.592000 12.888000 0.000000
0.000000 4.296000 0.000000
8.592000 4.296000 2.404000
0.000000 12.888000 2.404000
12.888000 8.592000 0.000000
4.296000 0.000000 2.404000
1.206000 7.398000 2.404000
7.398000 11.700000 2.404000
5.484000 1.206000 2.404000
11.700000 7.398000 2.404000
11.700000 7.398000 0.000000
5.484000 1.206000 0.000000
7.398000 11.700000 0.000000
1.206000 7.398000 0.000000
1.206000 5.484000 2.404000
7.398000 5.484000 2.404000"""

Fe_candidate_positions = np.array([list(map(float, line.split())) for line in Fe_coords_text.strip().split('\n')])
print(f"Fe candidate positions: {Fe_candidate_positions.shape}")

# Coordenadas de átomos de Nd (16 posiciones fijas)
Nd_coords_text = """8.592000 8.592000 0.000000
0.000000 8.592000 2.404000
8.592000 0.000000 2.404000
0.000000 0.000000 0.000000
12.888000 4.296000 0.000000
4.296000 4.296000 2.404000
12.888000 12.888000 2.404000
4.296000 12.888000 0.000000
4.296000 12.888000 2.404000
12.888000 12.888000 0.000000
4.296000 4.296000 0.000000
12.888000 4.296000 2.404000
0.000000 0.000000 2.404000
8.592000 0.000000 0.000000
0.000000 8.592000 0.000000
8.592000 8.592000 2.404000"""

Nd_positions = np.array([list(map(float, line.split())) for line in Nd_coords_text.strip().split('\n')])
print(f"Nd positions: {Nd_positions.shape}")

# Validación dinámica
n_Fe = len(Fe_candidate_positions)
n_Nd = len(Nd_positions)
assert n_Nd == 16, f"ERROR: Se esperan 16 posiciones de Nd, pero se encontraron {n_Nd}"
assert n_Fe >= 8, f"ERROR: Se necesitan al menos 8 posiciones candidatas para colocar 8 Ti"

print(f"\n✓ Coordenadas cargadas correctamente:")
print(f"  - {n_Fe} posiciones candidatas (Fe/Ti)")
print(f"  - {n_Nd} posiciones fijas (Nd)")
print(f"  - Total sistema: {n_Fe + n_Nd} átomos ({n_Fe - 8} Fe + 8 Ti + {n_Nd} Nd)")

# Advertencia si no son 96
if n_Fe != 96:
    print(f"\n⚠️  ADVERTENCIA: Se esperaban 96 posiciones de Fe, pero se encontraron {n_Fe}")
    print(f"   El sistema funcionará con {n_Fe} posiciones, pero verifica que las coordenadas sean correctas")

Fe candidate positions: (96, 3)
Nd positions: (16, 3)

✓ Coordenadas cargadas correctamente:
  - 96 posiciones candidatas (Fe/Ti)
  - 16 posiciones fijas (Nd)
  - Total sistema: 112 átomos (88 Fe + 8 Ti + 16 Nd)


---
## Sección 2: Visualización de Estructura Cristalina Inicial

In [51]:
# Visualización 3D interactiva de la estructura cristalina completa
fig = go.Figure()

# Añadir átomos de Fe candidatos (azul claro)
fig.add_trace(go.Scatter3d(
    x=Fe_candidate_positions[:, 0],
    y=Fe_candidate_positions[:, 1],
    z=Fe_candidate_positions[:, 2],
    mode='markers',
    name=f'Fe candidatos (n={len(Fe_candidate_positions)})',
    marker=dict(
        size=5,
        color='lightblue',
        opacity=0.7,
        line=dict(color='navy', width=1)
    ),
    hovertemplate='<b>Fe</b><br>X: %{x:.3f} Å<br>Y: %{y:.3f} Å<br>Z: %{z:.3f} Å<extra></extra>'
))

# Añadir átomos de Nd (dorado)
fig.add_trace(go.Scatter3d(
    x=Nd_positions[:, 0],
    y=Nd_positions[:, 1],
    z=Nd_positions[:, 2],
    mode='markers',
    name=f'Nd (n={len(Nd_positions)})',
    marker=dict(
        size=12,
        color='gold',
        opacity=0.95,
        line=dict(color='orange', width=2)
    ),
    hovertemplate='<b>Nd</b><br>X: %{x:.3f} Å<br>Y: %{y:.3f} Å<br>Z: %{z:.3f} Å<extra></extra>'
))

# Configurar layout
fig.update_layout(
    title=dict(
        text='Estructura Cristalina 3D: NdFe₁₂ (Supercelda 2×2×1)',
        font=dict(size=18, family='Arial')
    ),
    scene=dict(
        xaxis=dict(title='X (Å)', backgroundcolor='white', gridcolor='lightgray'),
        yaxis=dict(title='Y (Å)', backgroundcolor='white', gridcolor='lightgray'),
        zaxis=dict(title='Z (Å)', backgroundcolor='white', gridcolor='lightgray'),
        aspectmode='cube'
    ),
    width=1000,
    height=800,
    showlegend=True,
    legend=dict(
        x=0.02,
        y=0.98,
        bgcolor='rgba(255,255,255,0.9)',
        bordercolor='black',
        borderwidth=1
    ),
    hovermode='closest'
)

fig.show()

# Estadísticas de la estructura
print("\n📊 Estadísticas de la Estructura Cristalina:")
print(f"   Rango X: [{Fe_candidate_positions[:, 0].min():.3f}, {Fe_candidate_positions[:, 0].max():.3f}] Å")
print(f"   Rango Y: [{Fe_candidate_positions[:, 1].min():.3f}, {Fe_candidate_positions[:, 1].max():.3f}] Å")
print(f"   Rango Z: [{Fe_candidate_positions[:, 2].min():.3f}, {Fe_candidate_positions[:, 2].max():.3f}] Å")

# Centroide del sistema
all_positions_temp = np.vstack([Fe_candidate_positions, Nd_positions])
centroid = all_positions_temp.mean(axis=0)
print(f"   Centroide: ({centroid[0]:.3f}, {centroid[1]:.3f}, {centroid[2]:.3f}) Å")


📊 Estadísticas de la Estructura Cristalina:
   Rango X: [0.000, 15.391] Å
   Rango Y: [0.000, 15.391] Å
   Rango Z: [0.000, 2.404] Å
   Centroide: (6.848, 6.584, 1.223) Å


### Tabla Resumen del Sistema

In [52]:
# Crear tabla resumen
import math

n_Fe_candidates = len(Fe_candidate_positions)
summary_data = {
    'Tipo de Átomo': ['Fe (candidatos)', 'Ti (a optimizar)', 'Nd (fijos)', 'Total'],
    'Cantidad': [n_Fe_candidates, 8, 16, n_Fe_candidates + 16],
    'Estado': ['Variable', 'Variable', 'Fijo', '-'],
    'Descripción': [
        'Posiciones candidatas para Fe o Ti',
        'Sustituyen a 8 Fe (optimización)',
        'Tierras raras (no cambian)',
        f'{n_Fe_candidates - 8} Fe + 8 Ti + 16 Nd'
    ]
}

df_summary = pd.DataFrame(summary_data)
display(df_summary)

print(f"\n🎯 Espacio de búsqueda: C({n_Fe_candidates},8) = {math.comb(n_Fe_candidates, 8):.2e} configuraciones posibles")

,Tipo de Átomo,Cantidad,Estado,Descripción
0,Fe (candidatos),96,Variable,Posiciones candidatas para Fe o Ti
1,Ti (a optimizar),8,Variable,Sustituyen a 8 Fe (optimización)
2,Nd (fijos),16,Fijo,Tierras raras (no cambian)
3,Total,112,-,88 Fe + 8 Ti + 16 Nd



🎯 Espacio de búsqueda: C(96,8) = 1.33e+11 configuraciones posibles


---
## Sección 3: Configuración Inicial Aleatoria

In [53]:
# Crear configuración inicial con seed fijo para reproducibilidad
SEED_INICIAL = 42

atom_types_init, all_positions_init, Ti_indices_init, Fe_indices_init = crear_configuracion_inicial_3d(
    Fe_candidate_positions,
    Nd_positions,
    seed=SEED_INICIAL
)

print(f"✓ Configuración inicial creada (seed={SEED_INICIAL})")
print(f"  - Tipos atómicos: {atom_types_init.shape}")
print(f"  - Posiciones totales: {all_positions_init.shape}")
print(f"  - Índices Ti: {Ti_indices_init}")
print(f"  - Número de Fe: {len(Fe_indices_init)}")
print(f"  - Número de Ti: {len(Ti_indices_init)}")
print(f"  - Número de Nd: 16")

✓ Configuración inicial creada (seed=42)
  - Tipos atómicos: (112,)
  - Posiciones totales: (112, 3)
  - Índices Ti: [33 42 69 73 77 79 80 94]
  - Número de Fe: 88
  - Número de Ti: 8
  - Número de Nd: 16


In [54]:
# Preparar parámetros del potencial de Morse
morse_params_array = preparar_morse_params_array()
print("✓ Parámetros de Morse preparados")
print(f"  Shape: {morse_params_array.shape} (tipos_i, tipos_j, [D0, alpha, r0])")

# Mostrar algunos parámetros
print("\n📋 Parámetros de Morse para pares atómicos:")
pares = [('Fe', 'Fe'), ('Fe', 'Nd'), ('Fe', 'Ti'), ('Nd', 'Nd'), ('Nd', 'Ti'), ('Ti', 'Ti')]
for tipo1, tipo2 in pares:
    D0, alpha, r0 = get_morse_params_by_names(tipo1, tipo2)
    print(f"  {tipo1}-{tipo2:2s}: D₀={D0:.4f}, α={alpha:.4f}, r₀={r0:.4f}")

✓ Parámetros de Morse preparados
  Shape: (3, 3, 3) (tipos_i, tipos_j, [D0, alpha, r0])

📋 Parámetros de Morse para pares atómicos:
  Fe-Fe: D₀=0.7640, α=1.5995, r₀=2.7361
  Fe-Nd: D₀=0.6036, α=1.6458, r₀=3.1880
  Fe-Ti: D₀=0.8162, α=1.4480, r₀=2.9140
  Nd-Nd: D₀=0.3120, α=0.9450, r₀=4.0920
  Nd-Ti: D₀=0.4964, α=1.4401, r₀=3.4309
  Ti-Ti: D₀=0.6540, α=1.2118, r₀=3.3476


In [55]:
# Calcular energía inicial
energia_inicial = compute_total_energy_3d(atom_types_init, all_positions_init, morse_params_array)
print(f"\n⚡ Energía inicial (configuración aleatoria): {energia_inicial:.6f}")


⚡ Energía inicial (configuración aleatoria): 59609.069191


In [56]:
# Visualizar configuración inicial con Ti destacados
fig_init = plot_configuration_3d_plotly(
    Fe_candidate_positions,
    Nd_positions,
    Ti_indices_init,
    energia_inicial,
    title="Configuración Inicial (Aleatoria)"
)
fig_init.show()

print("\n✓ Configuración inicial visualizada (8 Ti en rojo, posiciones aleatorias)")


✓ Configuración inicial visualizada (8 Ti en rojo, posiciones aleatorias)


---
## Sección 4: Optimización con Simulated Annealing

### Parámetros del Algoritmo

**Enfriamiento Logarítmico:** $T(t) = \frac{c}{\log(t + t_0)}$

- **c = 100**: Constante de enfriamiento
- **t₀ = 10**: Offset temporal
- **T(0)** = c/ln(t₀) ≈ 43.4 (temperatura inicial)
- **Iteraciones**: 1,000,000 por run
- **Runs independientes**: 64 (con paralelización)

Este esquema de enfriamiento garantiza **convergencia global** al óptimo (Teorema de Hajek, 1988).

In [57]:
# Parámetros de optimización
N_RUNS = 64
MAX_ITERATIONS = 1_000_000
C_COOLING = 100.0
T0_COOLING = 10
N_JOBS = -1  # Usar todos los cores disponibles

print("🔧 Parámetros de Simulated Annealing:")
print(f"  - Runs independientes: {N_RUNS}")
print(f"  - Iteraciones por run: {MAX_ITERATIONS:,}")
print(f"  - Enfriamiento: T(t) = {C_COOLING}/log(t + {T0_COOLING})")
print(f"  - Temperatura inicial: {C_COOLING / np.log(T0_COOLING):.2f}")
print(f"  - Paralelización: {N_JOBS} (todos los cores)")
print(f"\n⏱️  Tiempo estimado: ~10-20 minutos (dependiendo de CPU)")

🔧 Parámetros de Simulated Annealing:
  - Runs independientes: 64
  - Iteraciones por run: 1,000,000
  - Enfriamiento: T(t) = 100.0/log(t + 10)
  - Temperatura inicial: 43.43
  - Paralelización: -1 (todos los cores)

⏱️  Tiempo estimado: ~10-20 minutos (dependiendo de CPU)


In [58]:
# Ejecutar optimización con múltiples runs en paralelo
print(f"\n🚀 Iniciando optimización: {N_RUNS} runs × {MAX_ITERATIONS:,} iteraciones...\n")
start_time = time.time()

resultados = ejecutar_multiples_runs_3d(
    n_runs=N_RUNS,
    Fe_candidate_positions=Fe_candidate_positions,
    Nd_positions=Nd_positions,
    c=C_COOLING,
    t0=T0_COOLING,
    max_iterations=MAX_ITERATIONS,
    morse_params_array=morse_params_array,
    n_jobs=N_JOBS,
    verbose=10
)

elapsed_time = time.time() - start_time
print(f"\n✓ Optimización completada en {elapsed_time:.2f} segundos ({elapsed_time/60:.2f} minutos)")
print(f"  Throughput: {N_RUNS * MAX_ITERATIONS / elapsed_time:.2e} iteraciones/segundo")


🚀 Iniciando optimización: 64 runs × 1,000,000 iteraciones...

Ejecutando 64 runs de Simulated Annealing 3D en paralelo...
  Parámetros: c=100.0, t₀=10, max_iter=1,000,000
  Jobs paralelos: -1 (todos los cores)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


ImportError: attempted relative import beyond top-level package

---
## Sección 5: Análisis de Resultados

In [ ]:
# Estadísticas agregadas de todos los runs
stats = get_run_statistics(resultados)

print("\n📊 Estadísticas de Todos los Runs:")
print(f"  Energía final:")
print(f"    - Mejor (mínima): {stats['energia_final_min']:.6f}")
print(f"    - Peor (máxima):  {stats['energia_final_max']:.6f}")
print(f"    - Media:          {stats['energia_final_mean']:.6f}")
print(f"    - Desv. Std:      {stats['energia_final_std']:.6f}")
print(f"\n  Mejora relativa (%):")
print(f"    - Mejor:  {stats['mejora_relativa_max']:.2f}%")
print(f"    - Peor:   {stats['mejora_relativa_min']:.2f}%")
print(f"    - Media:  {stats['mejora_relativa_mean']:.2f}%")
print(f"\n  Tasa de aceptación (%):")
print(f"    - Media: {stats['acceptance_rate_mean']:.2f}%")
print(f"    - Rango: [{stats['acceptance_rate_min']:.2f}%, {stats['acceptance_rate_max']:.2f}%]")

In [ ]:
# Obtener el mejor run
best_run = get_best_run(resultados)

print(f"\n🏆 Mejor Configuración Encontrada:")
print(f"  Run ID: {best_run['run_id']}")
print(f"  Energía inicial:  {best_run['energia_inicial']:.6f}")
print(f"  Energía final:    {best_run['energia_final']:.6f}")
print(f"  Mejora absoluta:  {best_run['energia_inicial'] - best_run['energia_final']:.6f}")
print(f"  Mejora relativa:  {best_run['mejora_relativa'] * 100:.2f}%")
print(f"  Iteración óptima: {best_run['iterations_to_best']:,}")
print(f"  Tasa aceptación:  {best_run['acceptance_rate']:.2f}%")

In [ ]:
# Top 10 mejores runs
top_runs = filter_top_runs(resultados, top_n=10)

# Crear tabla comparativa
top_df = pd.DataFrame([
    {
        'Rank': i + 1,
        'Run ID': r['run_id'],
        'Energía Final': f"{r['energia_final']:.6f}",
        'Mejora (%)': f"{r['mejora_relativa'] * 100:.2f}",
        'Iter Óptima': f"{r['iterations_to_best']:,}",
        'Aceptación (%)': f"{r['acceptance_rate']:.2f}"
    }
    for i, r in enumerate(top_runs)
])

print("\n🔝 Top 10 Mejores Runs:")
display(top_df)

---
## Sección 6: Visualizaciones

### 6.1 Configuración Óptima (3D Interactivo)

In [ ]:
# Visualización 3D de la configuración óptima
fig_optimo = plot_configuration_3d_plotly(
    Fe_candidate_positions,
    Nd_positions,
    best_run['Ti_indices'],
    best_run['energia_final'],
    title=f"Configuración Óptima (Run {best_run['run_id']})"
)
fig_optimo.show()

print(f"\n✓ Configuración óptima: 8 Ti (rojo) optimizados por SA")
print(f"  Índices Ti: {best_run['Ti_indices']}")

### 6.2 Proyecciones 2D (Comparación con Skelland)

In [ ]:
# Proyecciones 2D para comparar con Figura 29a de Skelland
fig_proj = plot_projections_2d(
    Fe_candidate_positions,
    Nd_positions,
    best_run['Ti_indices'],
    best_run['energia_final'],
    figsize=(18, 6)
)
plt.show()

print("\n✓ Proyecciones 2D (XY, XZ, YZ) para análisis visual")

### 6.3 Evolución de Energía

In [ ]:
# Evolución de energía del mejor run
fig_energy = plot_energy_evolution(
    best_run['energy_history'],
    title=f"Evolución de Energía - Mejor Run (ID {best_run['run_id']})",
    figsize=(14, 6)
)
plt.show()

print(f"\n✓ Evolución muestra convergencia: óptimo encontrado en iteración {best_run['iterations_to_best']:,}")

### 6.4 Comparación de Múltiples Runs

In [ ]:
# Comparación visual de todos los runs
fig_comparison = plot_multiple_runs_comparison_3d(
    resultados,
    top_n=10,
    figsize=(16, 10)
)
plt.show()

print("\n✓ Comparación de 64 runs: distribución de energías y convergencia")

---
## Sección 7: Análisis Espacial 3D

### Métricas de Skelland

Según Skelland et al., la configuración óptima debe cumplir:

1. **Distancia Ti-Nd promedio > 5 Å**: Los Ti deben estar alejados de los Nd
2. **Dispersión de Ti**: Los Ti deben estar distribuidos homogéneamente (no agrupados)
3. **Clustering score < 0.3**: Confirmación cuantitativa de dispersión

In [ ]:
# Calcular todas las métricas espaciales
# El índice de inicio de Nd es dinámico: n_Fe_candidates (90 o 96 según las coordenadas cargadas)
n_Fe_candidates = len(Fe_candidate_positions)
patron_optimo = analizar_patron_espacial_3d(
    best_run['all_positions'],
    best_run['Ti_indices'],
    Nd_start_idx=n_Fe_candidates
)

print("\n📐 Análisis Espacial de Configuración Óptima:\n")
print("=" * 60)
print("1. DISTANCIAS Ti-Nd (Métrica clave de Skelland):")
print(f"   Promedio: {patron_optimo['dist_Ti_Nd_promedio']:.3f} Å")
print(f"   Mínima:   {patron_optimo['dist_Ti_Nd_min']:.3f} Å")
print(f"   Máxima:   {patron_optimo['dist_Ti_Nd_max']:.3f} Å")
print(f"   Std Dev:  {patron_optimo['dist_Ti_Nd_std']:.3f} Å")

print("\n2. DISTANCIAS Ti-Ti (Homogeneidad):")
print(f"   Promedio (28 pares): {patron_optimo['dist_Ti_Ti_promedio']:.3f} Å")
print(f"   Mínima:              {patron_optimo['dist_Ti_Ti_min']:.3f} Å")
print(f"   Máxima:              {patron_optimo['dist_Ti_Ti_max']:.3f} Å")
print(f"   Std Dev:             {patron_optimo['dist_Ti_Ti_std']:.3f} Å")

print("\n3. VECINOS MÁS CERCANOS Ti-Ti:")
print(f"   Promedio: {patron_optimo['dist_Ti_vecino_promedio']:.3f} Å")
print(f"   Mínima:   {patron_optimo['dist_Ti_vecino_min']:.3f} Å")
print(f"   Máxima:   {patron_optimo['dist_Ti_vecino_max']:.3f} Å")

print("\n4. CLUSTERING (Dispersión vs Agrupamiento):")
print(f"   Score: {patron_optimo['clustering_score']:.4f}")
print(f"   Interpretación: {'DISPERSO ✓' if patron_optimo['clustering_score'] < 0.3 else 'AGRUPADO' if patron_optimo['clustering_score'] > 0.7 else 'INTERMEDIO'}")

print("\n5. DISTANCIA AL CENTROIDE:")
print(f"   Promedio: {patron_optimo['dist_centroide_promedio']:.3f} Å")
print(f"   Máxima:   {patron_optimo['dist_centroide_max']:.3f} Å")
print("=" * 60)

In [ ]:
# Visualización de métricas espaciales
fig_metrics = plot_spatial_metrics_3d(
    patron_optimo,
    figsize=(16, 5)
)
plt.show()

print("\n✓ Métricas espaciales visualizadas")

In [ ]:
# Interpretación del patrón espacial
interpretacion = interpretar_patron_3d(patron_optimo)
print("\n" + "="*70)
print("📝 INTERPRETACIÓN DEL PATRÓN ESPACIAL")
print("="*70)
print(interpretacion)
print("="*70)

### Comparación con Skelland et al.

**Conclusiones del paper (Figura 29a):**
- Los Ti tienden a estar **alejados de los Nd** (dist > 5 Å)
- Los Ti se distribuyen de forma **dispersa** (no agrupados)
- Configuración energéticamente favorable minimiza interacciones Ti-Nd

**Nuestros resultados:**

In [ ]:
# Crear tabla de comparación
comparison_data = {
    'Criterio': [
        'Dist Ti-Nd promedio',
        'Clustering score',
        'Patrón espacial'
    ],
    'Skelland et al.': [
        '> 5 Å (alejados)',
        '< 0.3 (dispersos)',
        'Distribución homogénea'
    ],
    'Nuestro Resultado': [
        f"{patron_optimo['dist_Ti_Nd_promedio']:.2f} Å",
        f"{patron_optimo['clustering_score']:.3f}",
        'Disperso ✓' if patron_optimo['clustering_score'] < 0.3 else 'Agrupado'
    ],
    'Cumple': [
        '✓' if patron_optimo['dist_Ti_Nd_promedio'] > 5.0 else '✗',
        '✓' if patron_optimo['clustering_score'] < 0.3 else '✗',
        '✓' if patron_optimo['clustering_score'] < 0.3 else '✗'
    ]
}

df_comparison = pd.DataFrame(comparison_data)
print("\n📊 Comparación con Skelland et al.:")
display(df_comparison)

---
## Sección 8: Validación

### 8.1 Validación del Cálculo Incremental de Energía

In [ ]:
# Validar que el cálculo incremental de ΔE coincide con el recalculo completo
print("🔍 Validación de compute_delta_E_swap_3d():\n")
print("Comparando cálculo incremental vs recalculo completo...\n")

# Realizar 10 swaps aleatorios y validar cada uno
np.random.seed(123)
n_tests = 10
errors = []

atom_types_test = best_run['atom_types'].copy()
Ti_indices_test = best_run['Ti_indices'].copy()
Fe_indices_test = best_run['Fe_indices'].copy()

for i in range(n_tests):
    # Proponer swap aleatorio
    ti_idx = np.random.randint(0, 8)
    fe_idx = np.random.randint(0, len(Fe_indices_test))
    
    # Validar
    delta_E_fast, delta_E_full, error_rel = validate_delta_E_3d(
        atom_types_test,
        best_run['all_positions'],
        Ti_indices_test,
        Fe_indices_test,
        ti_idx,
        fe_idx,
        morse_params_array
    )
    
    errors.append(abs(error_rel))
    print(f"Test {i+1:2d}: ΔE_fast={delta_E_fast:+.8f}, ΔE_full={delta_E_full:+.8f}, Error={error_rel:.2e}")

print(f"\n📊 Estadísticas de error:")
print(f"  Error promedio: {np.mean(errors):.2e}")
print(f"  Error máximo:   {np.max(errors):.2e}")
print(f"  Error std:      {np.std(errors):.2e}")

if np.max(errors) < 1e-10:
    print(f"\n✅ VALIDACIÓN EXITOSA: Cálculo incremental es exacto (error < 1e-10)")
else:
    print(f"\n⚠️  ADVERTENCIA: Error detectado en cálculo incremental")

### 8.2 Verificación de Convergencia

In [ ]:
# Verificar que la energía se estabiliza al final
energy_history = best_run['energy_history']
last_10_percent = int(0.1 * len(energy_history))
final_energies = energy_history[-last_10_percent:]

energy_std_final = np.std(final_energies)
energy_mean_final = np.mean(final_energies)

print("\n📈 Análisis de Convergencia:\n")
print(f"Últimas {last_10_percent:,} iteraciones (10% final):")
print(f"  Energía media:  {energy_mean_final:.6f}")
print(f"  Desviación std: {energy_std_final:.8f}")
print(f"  Rango: [{final_energies.min():.6f}, {final_energies.max():.6f}]")

# Criterio de convergencia: std < 0.01% de la energía
convergence_threshold = abs(energy_mean_final) * 0.0001
is_converged = energy_std_final < convergence_threshold

if is_converged:
    print(f"\n✅ CONVERGENCIA CONFIRMADA: Energía estable en fase final")
    print(f"   (std={energy_std_final:.2e} < threshold={convergence_threshold:.2e})")
else:
    print(f"\n⚠️  Energía aún fluctúa (puede necesitar más iteraciones)")

### 8.3 Robustez: Múltiples Seeds

In [ ]:
# Comparar dispersión de energías finales entre runs
energias_finales = [r['energia_final'] for r in resultados]
energia_min = min(energias_finales)
energia_max = max(energias_finales)
energia_std = np.std(energias_finales)

# Contar cuántos runs están cerca del óptimo (dentro de 1%)
threshold_1_percent = abs(energia_min) * 0.01
runs_near_optimal = sum(1 for e in energias_finales if abs(e - energia_min) < threshold_1_percent)

print(f"\n🎲 Análisis de Robustez ({N_RUNS} runs con diferentes seeds):\n")
print(f"Energía mínima encontrada: {energia_min:.6f}")
print(f"Energía máxima encontrada: {energia_max:.6f}")
print(f"Rango:                     {energia_max - energia_min:.6f}")
print(f"Desviación estándar:       {energia_std:.6f}")
print(f"\nRuns dentro de 1% del óptimo: {runs_near_optimal}/{N_RUNS} ({100*runs_near_optimal/N_RUNS:.1f}%)")

if runs_near_optimal > N_RUNS * 0.5:
    print(f"\n✅ ROBUSTEZ ALTA: >50% de runs convergen cerca del óptimo")
elif runs_near_optimal > N_RUNS * 0.2:
    print(f"\n✓ Robustez aceptable: ~{100*runs_near_optimal/N_RUNS:.0f}% converge cerca del óptimo")
else:
    print(f"\n⚠️  Robustez baja: solo {100*runs_near_optimal/N_RUNS:.0f}% converge cerca del óptimo")

---
## Resumen Final

In [ ]:
import math

n_Fe_candidates = len(Fe_candidate_positions)
print("\n" + "="*80)
print("🎯 RESUMEN FINAL - OPTIMIZACIÓN 3D NdFe₁₂ con Ti")
print("="*80)

print(f"\n1. CONFIGURACIÓN DEL PROBLEMA:")
print(f"   - Sistema: {n_Fe_candidates - 8} Fe + 8 Ti + 16 Nd = {n_Fe_candidates + 16} átomos")
print(f"   - Espacio de búsqueda: {math.comb(n_Fe_candidates, 8):.2e} configuraciones")
print(f"   - Método: Simulated Annealing con enfriamiento logarítmico")

print(f"\n2. PARÁMETROS DE OPTIMIZACIÓN:")
print(f"   - Runs independientes: {N_RUNS}")
print(f"   - Iteraciones por run: {MAX_ITERATIONS:,}")
print(f"   - Tiempo total: {elapsed_time/60:.2f} minutos")
print(f"   - Throughput: {N_RUNS * MAX_ITERATIONS / elapsed_time:.2e} iter/s")

print(f"\n3. MEJOR RESULTADO ENCONTRADO:")
print(f"   - Run ID: {best_run['run_id']}")
print(f"   - Energía inicial:  {best_run['energia_inicial']:.6f}")
print(f"   - Energía óptima:   {best_run['energia_final']:.6f}")
print(f"   - Mejora absoluta:  {best_run['energia_inicial'] - best_run['energia_final']:.6f}")
print(f"   - Mejora relativa:  {best_run['mejora_relativa']*100:.2f}%")
print(f"   - Índices Ti: {best_run['Ti_indices']}")

print(f"\n4. ANÁLISIS ESPACIAL (Métricas de Skelland):")
print(f"   - Dist Ti-Nd promedio: {patron_optimo['dist_Ti_Nd_promedio']:.3f} Å {'✓' if patron_optimo['dist_Ti_Nd_promedio'] > 5.0 else '✗'} (esperado > 5 Å)")
print(f"   - Dist Ti-Ti promedio: {patron_optimo['dist_Ti_Ti_promedio']:.3f} Å")
print(f"   - Clustering score:    {patron_optimo['clustering_score']:.4f} {'✓' if patron_optimo['clustering_score'] < 0.3 else '✗'} (esperado < 0.3)")

print(f"\n5. VALIDACIÓN:")
print(f"   - Cálculo incremental: {'✅ Exacto' if np.max(errors) < 1e-10 else '⚠️ Error'}")
print(f"   - Convergencia final:  {'✅ Estable' if is_converged else '⚠️ Fluctuante'}")
print(f"   - Robustez (seeds):    {'✅ Alta' if runs_near_optimal > N_RUNS * 0.5 else '✓ Aceptable' if runs_near_optimal > N_RUNS * 0.2 else '⚠️ Baja'}")

print(f"\n6. CONCLUSIÓN:")
if patron_optimo['dist_Ti_Nd_promedio'] > 5.0 and patron_optimo['clustering_score'] < 0.3:
    print("   ✅ Los resultados son CONSISTENTES con Skelland et al.")
    print("   ✅ Ti dispersos y alejados de Nd (configuración energéticamente favorable)")
else:
    print("   ⚠️  Los resultados difieren parcialmente de Skelland et al.")
    print("   → Puede requerir más iteraciones o ajuste de parámetros")

print("\n" + "="*80)
print("✓ Análisis completado")
print("="*80)